In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd

import json
import keras
import tensorflow as tf
from keras import layers, Model
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

Using TensorFlow backend.


In [2]:
csv_path = './UrbanSound8K/metadata/UrbanSound8K.csv'
spectrograms_path = "./numpySpectrograms/"
model_save_path = "basemodel-known"
test_size = 0.2
val_size = 0.2
batch_size = 16
num_classes = 5

In [3]:
class SpecLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, spec_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs), np.array(batch_y)

In [4]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < 5]
data_df_unknown = data_df.loc[data_df["classID"] >= 5]

X_trainval, X_test, y_trainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)
X_trash, X_unknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)
y_unknown = [-1] * len(y_unknown)

X_known_test = X_test
y_known_test = y_test
known_test_loader = SpecLoader(X_test, y_test, batch_size, spectrograms_path)
X_test = X_test + X_unknown
y_test = y_test + y_unknown
train_loader = SpecLoader(X_train, y_train, batch_size, spectrograms_path)
test_loader = SpecLoader(X_test, y_test, batch_size, spectrograms_path)
val_loader = SpecLoader(X_val, y_val, batch_size, spectrograms_path)
trainval_loader = SpecLoader(X_trainval, y_trainval, batch_size, spectrograms_path)

In [5]:
densenet = DenseNet201(
            include_top=False,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling="avg")
model = Sequential()
model.add(densenet)
model.add(layers.Dense(num_classes, activation="softmax"))
model.summary()
model.compile(
      optimizer="Adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 1920)              18321984  
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 9605      
Total params: 18,331,589
Trainable params: 18,102,533
Non-trainable params: 229,056
_________________________________________________________________


In [6]:
earlystopping = EarlyStopping(
                    patience=5, 
                    restore_best_weights=True)
checkpoint = ModelCheckpoint(
                    model_save_path, 
                    monitor="val_accuracy", 
                    save_best_only=True)

model.fit(x=train_loader,
          validation_data=val_loader,
          callbacks=[checkpoint, earlystopping],
          epochs=70,
          verbose=1
         )

Epoch 1/70
178/178 [==============================] - 180s 1s/step - loss: 0.7785 - accuracy: 0.7382 - val_loss: 15.3976 - val_accuracy: 0.4118
Epoch 2/70
178/178 [==============================] - 50s 283ms/step - loss: 0.4679 - accuracy: 0.8483 - val_loss: 0.1058 - val_accuracy: 0.8688
Epoch 3/70
178/178 [==============================] - 50s 283ms/step - loss: 0.3411 - accuracy: 0.8800 - val_loss: 0.0637 - val_accuracy: 0.8858
Epoch 4/70
178/178 [==============================] - 51s 285ms/step - loss: 0.2386 - accuracy: 0.9143 - val_loss: 0.3492 - val_accuracy: 0.8632
Epoch 5/70
178/178 [==============================] - 50s 282ms/step - loss: 0.2107 - accuracy: 0.9255 - val_loss: 0.0042 - val_accuracy: 0.9055
Epoch 6/70
178/178 [==============================] - 50s 283ms/step - loss: 0.1638 - accuracy: 0.9425 - val_loss: 0.0363 - val_accuracy: 0.8463
Epoch 7/70
178/178 [==============================] - 50s 282ms/step - loss: 0.1407 - accuracy: 0.9499 - val_loss: 0.0743 - val_acc

In [6]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))

In [7]:
model.load_weights(model_save_path)
# test_loss, test_accuracy= model.evaluate(x=known_test_loader)
y_predicted = model.predict_classes(x=known_test_loader, batch_size=None)

In [16]:
evaluate(y_predicted, y_known_test)
for average in ["macro", "weighted", "micro"]:
    f1 = f1_score(y_known_test, y_predicted, average=average)
    print(f1)

Overall accuracy: 0.945823927765237
Accuracy for class 2: 0.9359605911330049
Accuracy for class 4: 0.9651162790697675
Accuracy for class 0: 0.9856459330143541
Accuracy for class 3: 0.8947368421052632
Accuracy for class 1: 0.972972972972973
0.9505503979575508
0.9456365533255318
0.945823927765237
